from: https://github.com/SadmanSakib93/Stratified-k-fold-cross-validation-Image-classification-keras/blob/master/stratified_K_fold_CV.ipynb

In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import warnings
import os
import shutil
from PIL import ImageFile
warnings.simplefilter('error', Image.DecompressionBombWarning)
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = 1000000000
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2023-04-18 05:19:40.049512: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 05:19:40.943472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
cwd = os.getcwd()
print("current working directory: ",cwd)

current working directory:  /workspaces/proposal-ta


In [3]:
"""from google.colab import drive
drive.mount('/content/drive')
!ls
import sys
# This is the path to where in google drive the code is stored!
root_path = '/content/drive/My Drive/'
sys.path.append(root_path)
"""

root_path = '/workspaces/proposal-ta/images/'

In [4]:
### UBAH NAMA FILE MODEL SESUAI PENGUJIAN DI SINI
dataset_folder_name = os.path.join(root_path, '')
source_files = []
class_labels = ['a', 'ba', 'ca', 'da', 'ga', 
                'ja', 'ka','la','ma', 'na', 
                'nga', 'nya','pa', 'ra', 'sa', 
                'ta', 'wa', 'ya']
X = []
Y = []

img_rows, img_cols = 120, 120 # input image dimensions
train_path = os.path.join(dataset_folder_name, 'train')
validation_path = os.path.join(dataset_folder_name, 'validation')
test_path = os.path.join(dataset_folder_name, 'test')

In [ ]:
# augmentasi data
# bisa jadi sebelum atau setelah split

In [5]:
def transfer_between_folders(source, dest, split_rate):
    """ Based on the split ratio this function moves some portion of the source folder to destination folder!

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    global source_files
    source_files = os.listdir(source)
    if(len(source_files) != 0):
        transfer_file_numbers = int(len(source_files)*split_rate)
        transfer_index = random.sample(
            range(0, len(source_files)), transfer_file_numbers)
        for each_index in transfer_index:
            shutil.move(os.path.join(source, str(source_files[each_index])), os.path.join(
                dest, str(source_files[each_index])))

    else:
        print("No file moved. Source empty!")


def transfer_all_class_between_folders(source, dest, split_rate):
    """ Transfer the files from source to dest for all the classes. This function calls the 'transfer_between_folders' to actually perform the transfer.

        Args:
            source: str
                Source folder's path
            dest: str
                Destination folder's path
            split_rate: float
                Ratio of files to move from source to dest location

    """
    for label in class_labels:
        transfer_between_folders(os.path.join(dataset_folder_name, source, label),
                                 os.path.join(
                                     dataset_folder_name, dest, label),
                                 split_rate)


def my_metrics(y_true, y_pred):
    """ Calculate accuracy, precision, and f1 score of the model's prediction with respect to true labels.

        Args:
            y_true: list/array
                All true class labels
            y_pred: list/array
                All predicted class labels

        Returns:
            accuracy: float
                Accuracy measure of the model
            precision: float
                Precision measure of the model
            f1_Score: float
                F1-score measure of the model

    """
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    f1_Score = f1_score(y_true, y_pred, average='weighted')
    print("Accuracy  : {}".format(accuracy))
    print("Precision : {}".format(precision))
    print("f1Score : {}".format(f1_Score))
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    return accuracy, precision, f1_Score

In [6]:
transfer_all_class_between_folders('test','train', 1.0)
#transfer_all_class_between_folders('validation','train', 1.0)

In [7]:
transfer_all_class_between_folders('train', 'test', 0.20)

In [8]:
def prepare_name_with_labels(folder_name, dataset_type='train'):
    """ Prepare the file names (X) and the class labels (Y) from folder location of images.

        Args:
            folder_name: str
                Source folder's path

    """
    source_files = os.listdir(os.path.join(dataset_folder_name, dataset_type, folder_name))
    y_label = 0
    for i in range(len(class_labels)):
        if(folder_name == class_labels[i]):
            y_label = i
    for val in source_files:
        X.append(val)
        Y.append(y_label)

In [9]:
# Organize file names and class labels in X and Y variables
for i in range(len(class_labels)):
    prepare_name_with_labels(class_labels[i])

In [10]:
X = np.asarray(X)
Y = np.asarray(Y)

In [11]:
# arsitektur
batch_size = 32
epoch = 15
num_of_channels = 3
number_of_class_labels = len(class_labels)

In [12]:
"""
    Note that, this model structure is a very basic one. 
    To achieve better performance, you should change the model structure and hyperparameters according to your needs and data. 
    So, optimize the structure of the model!
"""

def get_model():
    activation_function = 'relu'

    model = Sequential()
    model.add(Conv2D(64, (3,3), padding='same',
                     activation=activation_function, input_shape=(img_rows, img_cols, num_of_channels)))
    model.add(Conv2D(64, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(32, (3,3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(32, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Conv2D(16, (3,3), padding='same',
                     activation=activation_function))
    model.add(Conv2D(16, (3,3), activation=activation_function))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    #model.add(Conv2D(64, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))
    
    #model.add(Conv2D(64, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    #model.add(Conv2D(128, (3,3), padding='same',
    #                 activation=activation_function))
    #model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation=activation_function))
    #model.add(Dropout(0.1))
    #model.add(Dense(32, activation=activation_function))
    #model.add(Dropout(0.1))
    #model.add(Dense(16, activation=activation_function))
    #model.add(Dropout(0.1))
    model.add(Dense(number_of_class_labels, activation='softmax'))

    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


model = get_model()
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 120, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 118, 118, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 64)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 59, 59, 32)        18464     
                                                                 
 conv2d_3 (Conv2D)           (None, 57, 57, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 32)       0         
 2D)                                                    

In [ ]:
"""model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 120, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    #tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(18, activation='softmax')
])

optimizer=tf.keras.optimizers.Adam(learning_rate=0.005)

model.compile(
    #optimizer='adam', 
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

model.summary()"""

In [13]:
# ===============Stratified K-Fold======================
skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, Y)
fold_num = 0
for train_index, val_index in skf.split(X, Y):
    # First cut all images from validation to train (if any exists)
    transfer_all_class_between_folders('validation', 'train'
                                       , 1.0)
    fold_num += 1
    print("Results for fold", fold_num)
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]
    # Move validation images of this fold from train folder to the validation folder
    for each_index in range(len(X_val)):
        class_label = ''
        for i in range(len(class_labels)):
            if(Y_val[each_index] == i):
                class_label = class_labels[i]
        # Then, copy the validation images to the validation folder
        shutil.move(os.path.join(dataset_folder_name, 'train', class_label, X_val[each_index]),
                    os.path.join(dataset_folder_name, 'validation', class_label, X_val[each_index]))

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.20,
        fill_mode="nearest")
    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    # Start ImageClassification Model
    train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = validation_datagen.flow_from_directory(
        validation_path,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        class_mode=None,  # only data, no labels
        shuffle=False)
    
    # fit model
    history = model.fit(train_generator,
                        epochs=epoch)

    predictions = model.predict(validation_generator, verbose=1)
    y_predictions = np.argmax(predictions, axis=1)
    true_classes = validation_generator.classes
    
    # evaluate validation performance
    print("***Performance on Validation data***")
    val_acc, val_prec, val_fScore = my_metrics(true_classes, y_predictions)

No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
No file moved. Source empty!
Results for fold 1
Found 979 images belonging to 18 classes.
Found 245 images belonging to 18 classes.
Epoch 1/15


2023-04-18 05:20:22.101808: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


31/31 [==============================] - 45s 1s/step - loss: 2.8933 - accuracy: 0.0531
Epoch 2/15
31/31 [==============================] - 43s 1s/step - loss: 2.4210 - accuracy: 0.2789
Epoch 3/15
31/31 [==============================] - 43s 1s/step - loss: 1.0405 - accuracy: 0.7017
Epoch 4/15
31/31 [==============================] - 43s 1s/step - loss: 0.6061 - accuracy: 0.8274
Epoch 5/15
31/31 [==============================] - 43s 1s/step - loss: 0.4894 - accuracy: 0.8427
Epoch 6/15
31/31 [==============================] - 42s 1s/step - loss: 0.3698 - accuracy: 0.8958
Epoch 7/15
31/31 [==============================] - 43s 1s/step - loss: 0.3123 - accuracy: 0.8989
Epoch 8/15
31/31 [==============================] - 42s 1s/step - loss: 0.2443 - accuracy: 0.9213
Epoch 9/15
31/31 [==============================] - 43s 1s/step - loss: 0.1711 - accuracy: 0.9459
Epoch 10/15
31/31 [==============================] - 42s 1s/step - loss: 0.1257 - accuracy: 0.9602
Epoch 11/15
31/31 [===========

2023-04-18 05:31:05.211426: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 2s 237ms/step
***Performance on Validation data***
Accuracy  : 0.8326530612244898
Precision : 0.8507907690127947
f1Score : 0.8345101154725271
[[12  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1  0]
 [ 0 10  0  0  3  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 0  0 12  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 11  0  0  0  0  0  1  0  0  0  2  0  0  0  0]
 [ 0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  0  0  0  1  0  0  0  0  2  0]
 [ 0  0  0  0  1  0 12  0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 10  0  0  0  0  1  0  2  0  0  0]
 [ 0  0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  1  0  0 12  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0 11  0  0  2  0  0  0  0]
 [ 0  4  0  0  0  0  0  0  0  0  0 10  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0 12  0  0  0  0  0]
 [ 0  0  1  0  0  0  1  0  0  0  0  0  1 11  0  0  0  0]
 [ 0  0  2  0  0  0  

...

31/31 [==============================] - 43s 1s/step - loss: 0.0167 - accuracy: 0.9939
Epoch 2/15
31/31 [==============================] - 43s 1s/step - loss: 0.0174 - accuracy: 0.9949
Epoch 3/15
31/31 [==============================] - 42s 1s/step - loss: 0.0159 - accuracy: 0.9949
Epoch 4/15
31/31 [==============================] - 42s 1s/step - loss: 0.0156 - accuracy: 0.9949
Epoch 5/15
 2/31 [>.............................] - ETA: 42s - loss: 0.0048 - accuracy: 1.0000

Bad pipe message: %s [b'\xbd\xf5\xae|\x02\x19\xa1\x87\x81\x94\x8c<\xefeYcb. q\xa3\x0bOL\xeb\xb7\xdfD\xfb"H\x01\xd1\xe1\x8a@U\xaa\xeb\xb7h:2\xd2\x04\xf7\xd5\xa4\x7f\xa0G\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00', b'\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01']
Bad pipe message: %s [b'\\\x8f\x1fN2\xf8G]\x0e\xb0\xb6\x91_cm4+\xad \x0e\x9ew\xad\x0c\xdb']
Bad pipe message: %s [b'Q\x1e\xd3\x02\xff\xc8\xa1>F\x9f\xa4\x81l\x12\xff?\xf8\xcaYS\x8a\xce^/\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00', b'\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x

31/31 [==============================] - 44s 1s/step - loss: 0.0152 - accuracy: 0.9949
Epoch 6/15
31/31 [==============================] - 42s 1s/step - loss: 0.0304 - accuracy: 0.9898
Epoch 7/15
31/31 [==============================] - 43s 1s/step - loss: 0.0284 - accuracy: 0.9929
Epoch 8/15
31/31 [==============================] - 42s 1s/step - loss: 0.0176 - accuracy: 0.9939
Epoch 9/15
31/31 [==============================] - 41s 1s/step - loss: 0.0091 - accuracy: 0.9969
Epoch 10/15
31/31 [==============================] - 42s 1s/step - loss: 0.0137 - accuracy: 0.9949
Epoch 11/15
31/31 [==============================] - 41s 1s/step - loss: 0.1272 - accuracy: 0.9765
Epoch 12/15
31/31 [==============================] - 41s 1s/step - loss: 0.0245 - accuracy: 0.9949
Epoch 13/15
31/31 [==============================] - 41s 1s/step - loss: 0.0109 - accuracy: 0.9969
Epoch 14/15
31/31 [==============================] - 41s 1s/step - loss: 0.0050 - accuracy: 0.9980
Epoch 15/15
31/31 [=======

2023-04-18 06:15:25.599012: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


8/8 [==============================] - 2s 227ms/step
***Performance on Validation data***
Accuracy  : 0.9959016393442623
Precision : 0.9961748633879781
f1Score : 0.9958903335217638
[[13  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 13  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 13  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 14  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 13  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 14  0  0  0  0]
 [ 0  0  0  0  0  0  

In [14]:
print("==============TEST RESULTS============")
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False 
)
predictions = model.predict(test_generator, verbose=1) 
y_predictions = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

test_acc, test_prec, test_fScore = my_metrics(true_classes, y_predictions)

==============TEST RESULTS============
Found 305 images belonging to 18 classes.


2023-04-18 06:17:23.660943: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


10/10 [==============================] - 3s 250ms/step
Accuracy  : 0.9278688524590164
Precision : 0.9320544530725722
f1Score : 0.9273168366144624
[[16  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0 13  0  0  0  2  0  0  0  0  1  1  0  0  0  0  0  0]
 [ 0  0 12  0  0  1  0  0  0  0  0  0  0  1  2  0  1  0]
 [ 0  0  0 16  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0 15  0  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 16  0  0  0  0  0  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0 16  0  0  0  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  1  0  0 16  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0 16  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0 16  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  1  0  0  0 16  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 17  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  2]

In [15]:
from sklearn.metrics import classification_report
clr = classification_report(true_classes, y_predictions,
                            labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], 
                            target_names=['a', 'ba','ca','da','ga','ja','ka','la','ma','na','nga','nya','pa','ra','sa','ta','wa','ya'])
print(clr)

              precision    recall  f1-score   support

           a       1.00      0.94      0.97        17
          ba       0.87      0.76      0.81        17
          ca       1.00      0.71      0.83        17
          da       1.00      0.94      0.97        17
          ga       1.00      1.00      1.00        16
          ja       0.79      0.88      0.83        17
          ka       0.94      1.00      0.97        17
          la       0.94      0.94      0.94        17
          ma       0.94      0.94      0.94        17
          na       1.00      0.94      0.97        17
         nga       0.94      0.94      0.94        17
         nya       0.94      0.94      0.94        17
          pa       0.94      0.94      0.94        17
          ra       0.85      1.00      0.92        17
          sa       0.83      0.88      0.86        17
          ta       1.00      1.00      1.00        17
          wa       0.94      0.94      0.94        17
          ya       0.85    

In [16]:
MODEL_FILENAME = root_path+"/modelpengujian20_3x3_arsitektur2.h5"
model.save(MODEL_FILENAME)